In [1]:
from collections import defaultdict
import numpy as np

In [23]:
def start_translation_chances(sentences):
    # Create a default dictionary for storing the translation probabilities
    translation_prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Initialize translation probabilities uniformly
    for pair in sentences:
        e, f = pair
        prob = 1 / len(f)
        for e_word in e:
            for f_word in f:
                translation_prob[e_word][f_word] = prob
    return translation_prob

In [24]:
def learn_translation_chances(sentences, repeat=10):
    # Initialize translation probabilities
    t = start_translation_chances(sentences)

    for i in range(repeat):
        print(f"Iteration {i+1}")

        # Initialize counts
        count = defaultdict(lambda: defaultdict(float))
        total = defaultdict(float)

        # E-step: Collect counts
        for e, f in sentences:
            s_total = defaultdict(float)
            for e_word in e:
                s_total[e_word] = 0
                for f_word in f:
                    s_total[e_word] += t[e_word][f_word]
            for e_word in e:
                for f_word in f:
                    increment = t[e_word][f_word] / s_total[e_word]
                    count[e_word][f_word] += increment
                    total[f_word] += increment
        # M-step: Update probabilities
        for f_word in total:
            for e_word in count:
                t[e_word][f_word] = count[e_word][f_word] / total[f_word]

        # Print s_total and counts for each iteration
        print(f"s_total: {dict(s_total)}")
        print(f"counts: {dict(count)}")
        print(f"total(f): {dict(total)}")
    return t

In [37]:
sentences = [
    (['dışarı','çıkalım','mı'], ['shall','we','go', 'out']),
    (['yemek','yiyelim'], ['Lets','have','dinner']),
    (['nerelisin'], ['where', 'are','you','from']),
    (['sıkıldım'], ['I', 'am','bored']),
    ( ['hava','nasıl'],['how', 'is','the','weather'])
]

translation_chances = learn_translation_chances(sentences)

Iteration 1
s_total: {'hava': 1.0, 'nasıl': 1.0}
counts: {'dışarı': defaultdict(<class 'float'>, {'shall': 0.25, 'we': 0.25, 'go': 0.25, 'out': 0.25, 'Lets': 0.0, 'have': 0.0, 'dinner': 0.0, 'where': 0.0, 'are': 0.0, 'you': 0.0, 'from': 0.0, 'I': 0.0, 'am': 0.0, 'bored': 0.0, 'how': 0.0, 'is': 0.0, 'the': 0.0, 'weather': 0.0}), 'çıkalım': defaultdict(<class 'float'>, {'shall': 0.25, 'we': 0.25, 'go': 0.25, 'out': 0.25, 'Lets': 0.0, 'have': 0.0, 'dinner': 0.0, 'where': 0.0, 'are': 0.0, 'you': 0.0, 'from': 0.0, 'I': 0.0, 'am': 0.0, 'bored': 0.0, 'how': 0.0, 'is': 0.0, 'the': 0.0, 'weather': 0.0}), 'mı': defaultdict(<class 'float'>, {'shall': 0.25, 'we': 0.25, 'go': 0.25, 'out': 0.25, 'Lets': 0.0, 'have': 0.0, 'dinner': 0.0, 'where': 0.0, 'are': 0.0, 'you': 0.0, 'from': 0.0, 'I': 0.0, 'am': 0.0, 'bored': 0.0, 'how': 0.0, 'is': 0.0, 'the': 0.0, 'weather': 0.0}), 'yemek': defaultdict(<class 'float'>, {'Lets': 0.3333333333333333, 'have': 0.3333333333333333, 'dinner': 0.3333333333333333, 'sha

In [38]:
from nltk.translate import IBMModel1, AlignedSent

aligned_text = [AlignedSent(pair[0], pair[1]) for pair in sentences]
ibm1 = IBMModel1(aligned_text, 10)

# Compare the translation probabilities
for e_word in translation_chances:
    for f_word in translation_chances[e_word]:
        print(f"Custom Implementation t({e_word}|{f_word}): {translation_chances[e_word][f_word]}")
        print(f"NLTK Implementation t({e_word}|{f_word}): {ibm1.translation_table[e_word][f_word]}")

Custom Implementation t(dışarı|shall): 0.3333333333333333
NLTK Implementation t(dışarı|shall): 0.3333333333333333
Custom Implementation t(dışarı|we): 0.3333333333333333
NLTK Implementation t(dışarı|we): 0.3333333333333333
Custom Implementation t(dışarı|go): 0.3333333333333333
NLTK Implementation t(dışarı|go): 0.3333333333333333
Custom Implementation t(dışarı|out): 0.3333333333333333
NLTK Implementation t(dışarı|out): 0.3333333333333333
Custom Implementation t(dışarı|Lets): 0.0
NLTK Implementation t(dışarı|Lets): 0.1111111111111111
Custom Implementation t(dışarı|have): 0.0
NLTK Implementation t(dışarı|have): 0.1111111111111111
Custom Implementation t(dışarı|dinner): 0.0
NLTK Implementation t(dışarı|dinner): 0.1111111111111111
Custom Implementation t(dışarı|where): 0.0
NLTK Implementation t(dışarı|where): 0.1111111111111111
Custom Implementation t(dışarı|are): 0.0
NLTK Implementation t(dışarı|are): 0.1111111111111111
Custom Implementation t(dışarı|you): 0.0
NLTK Implementation t(dışarı|y

In [39]:
# Record the output of the last iteration
print("Translation Probabilities After Last Iteration:")
for e_word in translation_chances:
    for f_word in translation_chances[e_word]:
        print(f"t({e_word}|{f_word}): {translation_chances[e_word][f_word]}")

Translation Probabilities After Last Iteration:
t(dışarı|shall): 0.3333333333333333
t(dışarı|we): 0.3333333333333333
t(dışarı|go): 0.3333333333333333
t(dışarı|out): 0.3333333333333333
t(dışarı|Lets): 0.0
t(dışarı|have): 0.0
t(dışarı|dinner): 0.0
t(dışarı|where): 0.0
t(dışarı|are): 0.0
t(dışarı|you): 0.0
t(dışarı|from): 0.0
t(dışarı|I): 0.0
t(dışarı|am): 0.0
t(dışarı|bored): 0.0
t(dışarı|how): 0.0
t(dışarı|is): 0.0
t(dışarı|the): 0.0
t(dışarı|weather): 0.0
t(çıkalım|shall): 0.3333333333333333
t(çıkalım|we): 0.3333333333333333
t(çıkalım|go): 0.3333333333333333
t(çıkalım|out): 0.3333333333333333
t(çıkalım|Lets): 0.0
t(çıkalım|have): 0.0
t(çıkalım|dinner): 0.0
t(çıkalım|where): 0.0
t(çıkalım|are): 0.0
t(çıkalım|you): 0.0
t(çıkalım|from): 0.0
t(çıkalım|I): 0.0
t(çıkalım|am): 0.0
t(çıkalım|bored): 0.0
t(çıkalım|how): 0.0
t(çıkalım|is): 0.0
t(çıkalım|the): 0.0
t(çıkalım|weather): 0.0
t(mı|shall): 0.3333333333333333
t(mı|we): 0.3333333333333333
t(mı|go): 0.3333333333333333
t(mı|out): 0.3333333